In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

HOST = "localhost"
PORT = 4223
SS_UID = "6RNni4"
LC_UID = 'Kgp'
import time

from tinkerforge.ip_connection import IPConnection
from tinkerforge.brick_silent_stepper import BrickSilentStepper
from tinkerforge.bricklet_load_cell_v2 import BrickletLoadCellV2
import numpy as np
from tqdm import tqdm
import pandas as pd
import time

ipcon = IPConnection() # Create IP connection
ss = BrickSilentStepper(SS_UID, ipcon) # Create device object
lc = BrickletLoadCellV2(LC_UID, ipcon) # Create device object
ipcon.connect(HOST, PORT) # Connect to brickd

In [7]:
PITCH = 2  # millimeters / turn
GEAR_RATIO = 1  # Assuming no gear ratio? https://www.omc-stepperonline.com/download/11HS12-0956D.pdf
MICRO_STEPS_PER_REV, RESOLUTION_SETTING = (
    256,
    ss.STEP_RESOLUTION_256,
)  # 256 micro-steps per macro-step
MACRO_STEPS_PER_REV = 200  # 256 macro-steps per revolution
STEPS_PER_REV = GEAR_RATIO * MACRO_STEPS_PER_REV * MICRO_STEPS_PER_REV
FINAL_DISTANCE = 30  # mm
MEASURE_FREQ = 0.5  # mm per measurement


MAX_VELOCITY = 0.5  # mm / s
MAX_STEPS_PER_SECOND = (
    MAX_VELOCITY * STEPS_PER_REV
) / PITCH  # Velocity: steps / sec
FINAL_STEPS = (FINAL_DISTANCE * STEPS_PER_REV) / PITCH
MEASURE_FREQ_IN_STEPS = (STEPS_PER_REV * MEASURE_FREQ) / PITCH

SAVE_FILENAME = "MUSCLE_1mm.csv"


In [8]:
df = pd.DataFrame(columns = ['Time', 'Force (newtons)', 'Distance (millimeters)'])

ss.enable() 
ss.set_current_position(0) # Initialize to 0
ss.set_motor_current(800) # 800 mA
ss.set_step_configuration(RESOLUTION_SETTING, True) # 1 / RESOLUTION_SETTING steps (interpolated)
ss.set_max_velocity(MAX_STEPS_PER_SECOND) # Velocity: steps/s
ss.set_speed_ramping(500, 5000) # Fast deacceleration (5000 steps/s^2) for stopping

# Initialize Tar & Set Stepper position to 0
tar = np.mean([lc.get_weight() for i in range(100)])
print(tar)

20042.0


In [9]:
# LETS RUN IT! NOTE: slapping on the negative sign sets the stepper to stretch
ss.set_steps(-FINAL_STEPS)
STARTING_TIME = time.time_ns() // 1_000_000

In [10]:
total_steps, total_distance, total_time, total_newtons, idx = 0, 0, 0, 0, 0

while 0.9 * FINAL_STEPS < total_steps or total_time < 80000:

    print(
        f"{idx} Total Distance {total_distance:.5f}, Total Force {total_newtons:.5f}, Total Time {total_time:.5f}"
    )
    total_steps = -ss.get_current_position()
    total_time = (time.time_ns() // 1_000_000) - STARTING_TIME
    total_distance = total_steps / STEPS_PER_REV * PITCH
    total_weight = tar - (lc.get_weight())  # grams

    # TODO: DOES THIS COME OUT IN GRAMS?
    total_newtons = - total_weight / 1000 * 9.80665  # m/s^2

    # display(f'{idx} Total Distance {total_distance:.5f}, Total Force {total_newtons:.5f}, Total Time {total_time:.5f}')

    df = df.append(
        {
            "Time": total_time,
            "Distance (millimeters)": total_distance,
            "Force (newtons)": total_newtons,
        },
        ignore_index=True,
    )
    time.sleep(0.01)
    # clear_output(wait=True)
    df.to_csv(SAVE_FILENAME)


0 Total Distance 0.00000, Total Force 0.00000, Total Time 0.00000
0 Total Distance 0.00004, Total Force -0.00000, Total Time 11.00000
0 Total Distance 0.00004, Total Force -0.00000, Total Time 35.00000
0 Total Distance 0.00008, Total Force -0.00000, Total Time 54.00000
0 Total Distance 0.00008, Total Force -0.00000, Total Time 72.00000
0 Total Distance 0.00012, Total Force -0.00000, Total Time 89.00000
0 Total Distance 0.00016, Total Force -0.00000, Total Time 105.00000
0 Total Distance 0.00020, Total Force -0.00000, Total Time 124.00000
0 Total Distance 0.00023, Total Force -0.00000, Total Time 140.00000
0 Total Distance 0.00027, Total Force -0.00000, Total Time 158.00000
0 Total Distance 0.00035, Total Force -0.00000, Total Time 174.00000
0 Total Distance 0.00039, Total Force -0.00000, Total Time 190.00000
0 Total Distance 0.00047, Total Force -0.00000, Total Time 205.00000
0 Total Distance 0.00055, Total Force -0.00000, Total Time 224.00000
0 Total Distance 0.00059, Total Force 0.00

KeyboardInterrupt: 

In [11]:
time.sleep(2) # Wait for motor to actually stop
ss.disable()
ipcon.disconnect()

In [ ]:
# df['Force (newtons)'] = - df['Force (newtons)']

In [12]:
import plotly.express as px
import matplotlib.pyplot as plt

# df['Distance (millimeters)'] = -df['Distance (millimeters)']
# df['Force (newtons))'] = -df['Force (newtons)']
fig = px.line(
    df,
    x="Distance (millimeters)",
    y="Force (newtons)",
    title=f"Stress / Strain of {SAVE_FILENAME}",
)
fig.show()


In [16]:
init_distance, init_force = 8.42, 0.598
final_distance, final_force = 14.696, 3.706
cross_sectional_area = (10 * 1) / 10**6 # mm^2 turned into m^2

strain = (final_distance - init_distance) / init_distance
stress = (final_force - init_force) / cross_sectional_area

youngs_modulus = stress / strain
print(f"Young's Modulus {(youngs_modulus / 1000)} kPa")

Young's Modulus 416.9751434034417 kPa
